In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

# Load the CSV file
data = pd.read_csv('output.csv')

# Split the data into features (X) and target variable (y)
X = data.drop('HORA CAMION', axis=1)  # Replace 'target_column' with the actual name of your target column
y = data['HORA CAMION']

# Label Encode the target variable if it is categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))  # Replace 16 with the desired number of neurons
model.add(Dense(1, activation='sigmoid'))  # Replace 1 with the number of output classes for multi-class classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')  # Replace 'binary_crossentropy' with the appropriate loss function for your problem

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Adjust the number of epochs and batch_size as needed

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100}%')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
X

,RUTA,FECHA,DIA SEMANA,HORA PREVIA,PARADA,ORIENTACION
0,231,10-06-2020,2,15:20:00,129,0
1,231,14-01-2023,5,12:56:00,76,0
2,231,25-09-2021,5,09:22:00,36,0
3,231,01-01-2023,6,00:00,8,0
4,231,17-12-2022,5,20:35:00,27,0
...,...,...,...,...,...,...
9995,231,16-05-2021,6,19:58:00,78,0
9996,231,25-05-2022,2,17:00:00,1,0
9997,231,22-10-2021,4,17:06:00,91,0
9998,231,15-08-2022,0,19:13:00,24,0


In [ ]:
import os
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPool2D
from keras.models import Model
import pandas as pd
import numpy as np


MOUNTPOINT = "/content/gdrive"
DATADIR = os.path.join(MOUNTPOINT, "MyDrive")
drive.mount(MOUNTPOINT)

class LeNet5(tf.keras.Model):
  def __init__(self):
    super(LeNet5, self).__init__()
    self.conv1 = Conv2D(filters=6, kernel_size=(2,2), padding="same", activation="relu")
    self.max_pool2x2 = MaxPool2D(pool_size=(2,2))
    self.conv2 = Conv2D(filters=16, kernel_size=(2,2), padding="same", activation="relu")
    self.flatten = Flatten()
    self.fc1 = Dense(units=120, activation="relu")
    self.fc2 = Dense(units=84, activation="relu")
    self.fc3=Dense(units=10, activation="softmax")
  def call(self, input_tensor):
    conv1 = self.conv1(input_tensor)
    maxpool1 = self.max_pool2x2(conv1)
    conv2 = self.conv2(maxpool1)
    maxpool2 = self.max_pool2x2(conv2)
    flatten = self.flatten(maxpool2)
    fc1 = self.fc1(flatten)
    fc2 = self.fc2(fc1)
    fc3 = self.fc3(fc2)
    return fc3

# Load the CSV file
data = pd.read_csv('dataset2.csv')

# Split the data into features (X) and target variable (y)




data[['dd', 'mm', 'yyyy']] = data['FECHA'].str.split('-', expand=True)

data['dd'] = data['dd'].astype(int)
data['mm'] = data['mm'].astype(int)
data['yyyy'] = data['yyyy'].astype(int)

data = data.drop('FECHA', axis=1)
data = data.drop('yyyy', axis=1)
data.astype(np.float32)

X = data.drop('HORA PREVIA', axis=1)
Y = data['HORA PREVIA']



# saving checkpoints
checkpoint_path = DATADIR + "/checkpoints/cp-epoch-{epoch}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
input_layer = Input(shape=(7))
model = LeNet5()(input_layer)
model = Model(inputs=input_layer, outputs=model)
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")

model.fit(x=X, y=Y, batch_size=256,
          epochs=5, callbacks=[cp_callback])

In [ ]:
import pandas as pd



# Print the resulting DataFrame
print(data)

2nd Try

In [ ]:
X

,RUTA,DIA SEMANA,HORA PREVIA,PARADA,ORIENTACION,dd,mm
0,231,2,0.638889,129,0,10,6
1,231,5,0.538889,76,0,14,1
2,231,5,0.390278,36,0,25,9
3,231,6,0.000000,8,0,1,1
4,231,5,0.857639,27,0,17,12
...,...,...,...,...,...,...,...
9995,231,6,0.831944,78,0,16,5
9996,231,2,0.708333,1,0,25,5
9997,231,4,0.712500,91,0,22,10
9998,231,0,0.800694,24,0,15,8


In [ ]:
def transformTime(times):

  hours = []
  for time in times:
    hours.append(time.split(':'))


  timeFraction = []
  for hour in hours:
    timeFraction.append(((int(hour[0]) * 60 * 60) + (int(hour[1]) * 60)) / (24*60*60) )

  return timeFraction


In [ ]:
# @title RUTA  | DIA SEMANA | HORA P | PARADA | ORIENTACION | DIA | MES -> RELU - HORA CAMION

# @markdown 7 - 10 - 1
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator

data = pd.read_csv('output.csv')

data[['dd', 'mm', 'yyyy']] = data['FECHA'].str.split('-', expand=True)

data['dd'] = data['dd'].astype(int)
data['mm'] = data['mm'].astype(int)
data['yyyy'] = data['yyyy'].astype(int)

data = data.drop('FECHA', axis=1)
data = data.drop('yyyy', axis=1)
data['HORA PREVIA'] = transformTime(data['HORA PREVIA'])
data['HORA CAMION'] = transformTime(data['HORA CAMION'])
data.astype(np.float32)

X = data.drop('HORA CAMION', axis=1)
Y = data['HORA CAMION']


model = Sequential()
model.add(Dense(10, input_shape=(7,), activation='relu'))
model.add(Dense(9, input_shape=(10,), activation='relu'))
model.add(Dense(8, input_shape=(9,), activation='relu'))
model.add(Dense(7, input_shape=(8,), activation='relu'))
model.add(Dense(6, input_shape=(7,), activation='relu'))
model.add(Dense(1))
# Compile the model
model.compile(optimizer='adam', loss='mse')

data_generator = TimeseriesGenerator(X, Y, length=10, batch_size=5)

model.fit(data_generator, epochs=50)

Epoch 1/50
1996/1996 [==============================] - 35s 17ms/step - loss: 185.1132
Epoch 2/50
1996/1996 [==============================] - 32s 16ms/step - loss: 0.1862
Epoch 3/50
1996/1996 [==============================] - 32s 16ms/step - loss: 0.0364
Epoch 4/50
1996/1996 [==============================] - 27s 14ms/step - loss: 0.0375
Epoch 5/50
1996/1996 [==============================] - 30s 15ms/step - loss: 0.0433
Epoch 6/50
1996/1996 [==============================] - 35s 18ms/step - loss: 0.0476
Epoch 7/50
1996/1996 [==============================] - 30s 15ms/step - loss: 0.0500
Epoch 8/50
1996/1996 [==============================] - 34s 17ms/step - loss: 0.0489
Epoch 9/50
1996/1996 [==============================] - 26s 13ms/step - loss: 0.0485
Epoch 10/50
1996/1996 [==============================] - 36s 18ms/step - loss: 0.0488
Epoch 11/50
1996/1996 [==============================] - 43s 22ms/step - loss: 0.0492
Epoch 12/50
1996/1996 [==============================] - 31s 

In [ ]:
# @title SECUENTIAL 4-6-2-1
# @markdown RUTA  | DIA SEMANA | HORA P | PARADA -> Linear - HORA CAMION
from keras import metrics

# @markdown 4-6-2-1
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator

data = pd.read_csv('output.csv')

#data[['dd', 'mm', 'yyyy']] = data['FECHA'].str.split('-', expand=True)

#data['dd'] = data['dd'].astype(int)
#data['mm'] = data['mm'].astype(int)
#data['yyyy'] = data['yyyy'].astype(int)
data = data.drop('RUTA', axis=1)
data = data.drop('FECHA', axis=1)
#data = data.drop('yyyy', axis=1)
data['HORA PREVIA'] = transformTime(data['HORA PREVIA'])
data['HORA CAMION'] = transformTime(data['HORA CAMION'])
data.astype(np.float32)

X = data.drop('HORA CAMION', axis=1)
X = X.drop('ORIENTACION', axis=1)
Y = data['HORA CAMION']


model = Sequential()
model.add(Dense(10, input_shape=(3,), activation='softplus'))
model.add(Dense(1))
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=metrics.BinaryAccuracy(name="Preciso", threshold=0.01))

while range(0,10):
  data_generator = TimeseriesGenerator(X, Y, length=20, batch_size=10)
  model.fit(data_generator, epochs=1)

652/998 [==================>...........] - ETA: 3s - loss: 0.0302 - Preciso: 0.0000e+00

In [ ]:
data['HORA CAMION']

0       15:40:00
1       13:16:00
2       09:42:00
3       07:03:00
4       20:55:00
          ...   
9995    20:18:00
9996    17:20:00
9997    17:26:00
9998    19:33:00
9999    12:24:00
Name: HORA CAMION, Length: 10000, dtype: object

In [ ]:
# The starting time point for prediction in HH:MM format
time_steps = 30  # The number of predictions to make
HoraI = 12
MinI = 33
SegI = 5
# Convert starting_time to numerical format
starting_time = 0.36 # Convert starting_time to numerical representation based on the same method used in step 2
output = []
# Initialize input_data for prediction
input_data = np.zeros((1,3))
#input_data[0,0] = 231
input_data[0,0] = 5
input_data[0,1] = ((HoraI * 60 * 60) + (MinI * 60) + SegI) / (60*60*24)
input_data[0,2] = 56

#input_data[0,6] = 2024


# Generate predictions for the remaining time steps
predict = model.predict(input_data)
segundos = predict[0,0] * 60 * 60 * 24
Hora = segundos // 3600
segundos = segundos % 3600
minutos = segundos // 60
segundos = segundos % 60
print(f' {HoraI}:{MinI}:{SegI} - {Hora.astype(int)}:{minutos.astype(int)}:{segundos.astype(int)}')
# Convert predictions back to HH:MM format

1/1 [==============================] - 0s 289ms/step
 12:33:5 - 14:43:32


In [ ]:

segundos = predict[0,0] * 60 * 60 * 24
Hora = segundos // 3600
segundos = segundos % 3600
minutos = segundos // 60
segundos = segundos % 60
print(f' {HoraI}:{MinI}:{SegI} - {Hora.astype(int)}:{minutos.astype(int)}:{segundos.astype(int)}')

In [ ]:

def get_all_times(start, end, spacing):
    start_time = datetime.strptime(start, '%H:%M')
    end_time = datetime.strptime(end, '%H:%M')

    times = []
    current_time = start_time

    while current_time <= end_time:
        times.append(current_time.strftime('%H:%M'))
        current_time += timedelta(minutes=spacing)

    return times


# Example Usage
start_time = "08:00"
end_time = "10:00"
spacing_minutes = 20

all_times = get_all_times(start_time, end_time, spacing_minutes)
print(all_times)

In [ ]:
X

,RUTA,DIA SEMANA,HORA PREVIA,PARADA,HORA CAMION
0,231,2,0.638889,129,0.652778
1,231,5,0.538889,76,0.552778
2,231,5,0.390278,36,0.404167
3,231,6,0.000000,8,0.293750
4,231,5,0.857639,27,0.871528
...,...,...,...,...,...
9995,231,6,0.831944,78,0.845833
9996,231,2,0.708333,1,0.722222
9997,231,4,0.712500,91,0.726389
9998,231,0,0.800694,24,0.814583
